In [ ]:
import os
import pandas as pd
import random
from copy import deepcopy
from collections import defaultdict
import geopandas as gp
from matplotlib import pyplot as plt
from pam import parse

path_to_repo = '/Users/.../PythonProjects/pam'

## Load Data
Here we load simple travel diary data of London commuters. This is a very simple 0.1% sample of data about work and education commutes from the 2011 census. Because we're sharing this date - we've aggregated locations to borough level and randomized personal attributes - so don't get too excited about the results.

The data is available in `pam/example_data`.

In [ ]:
trips = pd.read_csv(os.path.join(path_to_repo, 'example_data', 'example_travel_diaries.csv'))
attributes = pd.read_csv(os.path.join(path_to_repo, 'example_data' , 'example_attributes.csv'))
attributes.set_index('pid', inplace=True)

In [ ]:
attributes.head()

In [ ]:
trips.head()

In [ ]:
trips['purp'].unique(), trips['mode'].unique()

## Imagine we had better data...

Conjure up additional data to aid demonstration.

We add:
- `age` to person attributes
- activities: 
    - `leisure`
    - `health`
    - `shopping`
    - `escort`

and we put people in shared households based on shared hzone (originally it's one person per household)

**Randomly.**

In [ ]:
import numpy as np

# add age column to attributes
attributes['age'] = [int(a) for a in np.random.normal(40.5, 10, len(attributes))]

In [ ]:
# add some extra activities to trips
zones = list(set(trips['hzone'].unique()) | set(trips['ozone'].unique()) | set(trips['dzone'].unique()))
purp = ['leisure', 'health', 'shopping', 'escort']
mode = ['pt', 'car', 'walk', 'bike']

def enrich_activities(group):
    new_group = pd.DataFrame(columns=group.columns)
    for idx in group.index:
        trip = group.loc[idx, :]
        try:
            next_start = group.loc[idx+1, :]['tst']
        except KeyError:
            next_start = 1439
        new_group = new_group.append(trip)
        # append a random activity
        activity_loc = random.choice(zones)
        activity_purp = random.choice(purp)
        activity_mode = random.choice(mode)

        act_times = [random.randint(trip['tet']+1, next_start-1) for i in range(4)]
        act_times.sort()

        new_group = new_group.append(pd.DataFrame(
            {'uid': ['{}_act_to'.format(trip['uid']), '{}_act_from'.format(trip['uid'])], 
             'pid': [trip['pid'], trip['pid']], 'hid': [trip['hid'], trip['hid']], 
             'seq': [0, 0], 'hzone': [trip['hzone'], trip['hzone']], 'ozone': [trip['dzone'], activity_loc], 
             'dzone': [activity_loc, trip['ozone']], 'purp': [activity_purp, activity_purp], 
             'mode': [activity_mode, activity_mode], 'tst': [act_times[0], act_times[2]], 
             'tet': [act_times[1], act_times[3]], 'freq': [trip['freq'], trip['freq']]}))
    new_group = new_group.reset_index(drop=True)
    new_group = new_group.drop(['seq'], axis=1)
    new_group = new_group.rename_axis('seq').reset_index()
    return new_group

new_trips = trips.groupby('pid').apply(enrich_activities).reset_index(drop=True)

In [ ]:
# generate some households (as opposed one person to a household)
# bunch up people who share the same hzone
import uuid

def bunch_up_people(group):
    group['pid'].unique()
    households = []
    i = 1
    household= []
    h_size = random.randint(1,5)
    for p in group['pid'].unique():
        household.append(p)
        i += 1

        if i>h_size:
            households.append(household)
            h_size = random.randint(1,5)
            household= []
            i = 1

    return dict(zip([str(uuid.uuid4()) for i in range(len(households))], households))

In [ ]:
households_map = {}
households_per_zone = new_trips.groupby('hzone').apply(bunch_up_people)
for idx in households_per_zone.index:
    households_map = {**households_map, **households_per_zone[idx]}

In [ ]:
# ze old switcheroo person to household id
new_keys = []
new_values = []
for key, value in households_map.items():
    for item in value:
        new_keys.append(item)
        new_values.append(key.lower())
person_to_household_map = dict(zip(new_keys, new_values))

In [ ]:
dict(list(person_to_household_map.items())[0:15])

In [ ]:
new_trips['hid'] = new_trips['pid'].map(person_to_household_map)

## Input data after changes 

In [ ]:
attributes.head()

In [ ]:
new_trips.head(10)

In [ ]:
new_trips.info()

In [ ]:
new_trips['purp'].unique(), new_trips['mode'].unique()

In [ ]:
def print_simple_stats(population):
    """
    Print some simple population statistics.
    """
    time_at_home = 0
    travel_time = 0 
    low_income_central_trips = 0
    high_income_central_trips = 0
    
    for hid, hh in population.households.items():
        for pid, person in hh.people.items():
            freq = person.freq
            
            for p in person.plan:
                if p.act == 'travel':
                    duration = p.duration.seconds * freq / 3600
                    travel_time += duration
                    
                    if p.end_location.area == "Westminster,City of London":
                        if person.attributes['inc'] == "low":
                            low_income_central_trips += freq
                            
                        elif person.attributes['inc'] == "high":
                            high_income_central_trips += freq
                    
                else:  # activity
                    if p.act == 'home':
                        duration = p.duration.seconds * freq / 3600
                        time_at_home += duration
                        
    print(f"Population total time at home: {time_at_home/1000000:.2f} million hours")
    print(f"Population total travel time: {travel_time/1000000:.2f} million hours")
    print(f"Low income trips to Central London: {low_income_central_trips} trips")
    print(f"High income trips to Central London: {high_income_central_trips} trips")

## Create the population

In [ ]:
population = parse.load_travel_diary(new_trips, attributes)

In [ ]:
rando_hh_id = list(households_map.keys())[0]
rando_pid_in_hh = list(population.households[rando_hh_id].people)[0]

In [ ]:
population.households[rando_hh_id].people

In [ ]:
population.households[rando_hh_id].people[rando_pid_in_hh].plan.print()

In [ ]:
person = population.households[rando_hh_id].people[rando_pid_in_hh]

In [ ]:
len(list(person.activities))

In [ ]:
population.households[rando_hh_id].people[rando_pid_in_hh].attributes

In [ ]:
print_simple_stats(population)

# PAM Simple Policies

Based on [link](https://docs.google.com/spreadsheets/d/1FQMa7dLe2cv1NEZnbu5cZo3v07tKXINwvOaLQYoEp-M/edit#gid=0)

In [ ]:
from pam import modify

def apply_policies(population, policies: list):

    new_population = deepcopy(population) 
    for hid, household in new_population.households.items():
        for policy in policies:
            policy.apply_to(household)
    return new_population

## Household Quarantine

Probabilistically apply quarantine to a household (remove all activities - stay at home) 

### Household-based

If you have a probability of a household having to be quarantined

In [ ]:
policy_household_quarantine_per_household = \
    modify.HouseholdQuarantined(
        modify.HouseholdProbability(0.01)
    )

In [ ]:
print_simple_stats(apply_policies(population, [policy_household_quarantine_per_household]))

### Person-based

If you have a probability of any one person living in the household having to be quarantined.

The probability of the household being quarantined is then $1 - (1-P)^n$, where $P$ is the probability any one person being quarantined and $n$ is the number of people in the household; $(1-P)^n$ is the probability of no one having to be quarantined.

In [ ]:
policy_household_quarantine_per_person = \
    modify.HouseholdQuarantined(
        modify.PersonProbability(0.01)
    )

In [ ]:
print_simple_stats(apply_policies(population, [policy_household_quarantine_per_person]))

(Should be equivalent to `RemoveActivity` removing all non-home activities with `policy_type='household'`, and `probability_level='person'`)

In [ ]:
policy_remove_any_education = \
    modify.HouseholdPolicy(
        modify.RemoveActivity(['work', 'leisure', 'shopping', 'health', 'education', 'escort']), 
        modify.PersonProbability(0.01)
)

print_simple_stats(apply_policies(population, [policy_household_quarantine_per_person]))

## Remove Higher Education

Remove all education activity for persons over age of 17

In [ ]:
def age_condition_over_17(attribute_value):
    return attribute_value > 17

policy_remove_higher_education = \
    modify.PersonPolicy(
        modify.RemoveActivity(['education']),
        modify.PersonProbability(1),
        modify.PersonAttributeFilter({'age': age_condition_over_17}, how='all')
)

In [ ]:
print_simple_stats(apply_policies(population, [policy_remove_higher_education]))

## Remove Education

Probabilistically remove education activities from a person

In [ ]:
policy_remove_any_education = \
    modify.HouseholdPolicy(
        modify.RemoveActivity(['education', 'escort']), 
        modify.PersonProbability(0.95)
)

In [ ]:
print_simple_stats(apply_policies(population, [policy_remove_any_education]))

#### Affecting different activities

You can choose to remove different activities to the ones that carry a probability. Here, education activities affect removal of both education and escort on a household level.

In [ ]:
policy_remove_any_education = \
    modify.HouseholdPolicy(
        modify.RemoveActivity(['education', 'escort']), 
        modify.ActivityProbability(['education'], 0.95)
)

In [ ]:
print_simple_stats(apply_policies(population, [policy_remove_any_education]))

## Remove Leisure Activities

Remove all leisure activities

In [ ]:
policy_remove_leisure = \
    modify.PersonPolicy(
        modify.RemoveActivity(['leisure']), 
        modify.PersonProbability(1)
)

In [ ]:
print_simple_stats(apply_policies(population, [policy_remove_leisure]))

## Remove Health Activities

Probabilistically remove **individual** health activities from a person

In [ ]:
policy_remove_health = \
    modify.ActivityPolicy(
        modify.RemoveActivity(['health']), 
        modify.ActivityProbability(['health'], 0.5)
)

In [ ]:
print_simple_stats(apply_policies(population, [policy_remove_health]))

## Unemployment/Furlough

Probabilistically remove all work activities from a person

In [ ]:
policy_unemployment_and_furlough = \
    modify.PersonPolicy(
        modify.RemoveActivity(['work']), 
        modify.PersonProbability(0.1)
)

In [ ]:
print_simple_stats(apply_policies(population, [policy_unemployment_and_furlough]))

## Work from Home

Probabilistically remove all work activities from a person

In [ ]:
policy_work_from_home = \
    modify.PersonPolicy(
        modify.RemoveActivity(['work']), 
        modify.PersonProbability(0.5)
)

In [ ]:
print_simple_stats(apply_policies(population, [policy_work_from_home]))

## Reduced Work Activity

Probabilistically remove **individual** work activities from a person

In [ ]:
policy_reduced_work_activity = \
    modify.ActivityPolicy(
        modify.RemoveActivity(['work']), 
        modify.ActivityProbability(['work'], 0.2)
)

In [ ]:
print_simple_stats(apply_policies(population, [policy_reduced_work_activity]))

## Remove Shopping 

Probabilistically remove **individual** shopping activities from a person

In [ ]:
policy_remove_shopping = \
    modify.ActivityPolicy(
        modify.RemoveActivity(['shop']), 
        modify.ActivityProbability(['shop'], 0.5)
)

In [ ]:
print_simple_stats(apply_policies(population, [policy_remove_shopping]))

## All together now!

In [ ]:
all_together_pop = apply_policies(
    population, 
    [policy_household_quarantine_per_person, 
     policy_remove_higher_education, 
     policy_remove_any_education, 
     policy_remove_leisure, 
     policy_remove_health, 
     policy_unemployment_and_furlough, 
     policy_work_from_home, 
     policy_reduced_work_activity, 
     policy_remove_shopping])

print_simple_stats(all_together_pop)